In [8]:
!pip install transformers torch sentencepiece

In [23]:
from transformers import pipeline
import json
import torch

class LocalInvestmentReportGenerator:
    def __init__(self):
        self.generator = pipeline(
            "text-generation",
            model="facebook/opt-350m",
            device=0 if torch.cuda.is_available() else -1
        )

    def generate_structured_report(self, client_data):
        report = f"""Investment Portfolio Summary Report

1. CLIENT PROFILE
Name: {client_data['profile']['name']}
Investment Goal: {client_data['profile']['investment_goal']}
Risk Tolerance: {client_data['profile']['risk_tolerance']}
Time Horizon: {client_data['profile']['time_horizon']}

2. PORTFOLIO PERFORMANCE
Total Portfolio Value: ${client_data['portfolio_metrics']['total_value']:,}
Year-to-Date Return: {client_data['portfolio_metrics']['ytd_return']}%

3. ASSET ALLOCATION
"""

        allocation = client_data['portfolio_metrics']['asset_allocation']
        for asset, percentage in allocation.items():
            report += f"\n{asset.title()}: {percentage}%"

        risk_metrics = client_data['portfolio_metrics']['risk_metrics']
        report += f"\n\n4. RISK ANALYSIS\n"
        report += f"\nSharpe Ratio: {risk_metrics['sharpe_ratio']}"
        report += f"\nVolatility: {risk_metrics['volatility']}%"
        report += f"\nMaximum Drawdown: {risk_metrics['max_drawdown']}%"
        benchmark = client_data['portfolio_metrics']['benchmark_comparison']
        report += f"\n\n5. BENCHMARK COMPARISON\n"
        report += f"\nS&P 500 Return: {benchmark['sp500_return']}%"
        report += f"\nRelative Performance: {benchmark['relative_performance']}%"
        report += "\n\n6. RECOMMENDATIONS\n"
        recommendations = self.generate_recommendations(client_data)
        report += f"\n{recommendations}"
        report += "\n\nDISCLAIMER\n\n"
        report += "Past performance does not guarantee future results. "
        report += "This report is for informational purposes only."

        return report

    def generate_recommendations(self, client_data):
        """Generate personalized recommendations based on client profile and performance"""
        profile = client_data['profile']
        metrics = client_data['portfolio_metrics']

        recommendations = []

        if profile['risk_tolerance'] == 'moderate' and metrics['asset_allocation']['stocks'] > 70:
            recommendations.append("Consider reducing equity exposure to better align with your moderate risk tolerance.")

        elif profile['risk_tolerance'] == 'aggressive' and metrics['asset_allocation']['stocks'] < 70:
            recommendations.append("Given your aggressive risk tolerance, consider increasing equity exposure for potentially higher returns.")

        if metrics['benchmark_comparison']['relative_performance'] < -2:
            recommendations.append("Portfolio is underperforming the benchmark. Review asset allocation and consider rebalancing.")

        if metrics['asset_allocation']['cash'] > 10:
            recommendations.append("High cash position may be a drag on returns. Consider deploying capital based on your investment strategy.")

        if profile['investment_goal'] == 'retirement' and profile['time_horizon'] == 'long_term':
            recommendations.append("Maintain current bond allocation for stability while keeping sufficient equity exposure for long-term growth.")

        if not recommendations:
            recommendations.append("Portfolio is well-aligned with your investment objectives. Continue regular monitoring and rebalancing as needed.")

        return "\n".join(f" {rec}" for rec in recommendations)

def main():
    try:
        with open('sample2.json') as file:
            data = json.load(file)

        generator = LocalInvestmentReportGenerator()

        for client in data['clients']:
            print(f"\nGenerating report for {client['profile']['name']}...")
            report = generator.generate_structured_report(client)
            print(report)
            print("\n" + "="*80 + "\n")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Device set to use cpu



Generating report for Pranav Raj...
Investment Portfolio Summary Report

1. CLIENT PROFILE
Name: Pranav Raj
Investment Goal: retirement
Risk Tolerance: moderate
Time Horizon: long_term

2. PORTFOLIO PERFORMANCE
Total Portfolio Value: $500,000
Year-to-Date Return: 8.5%

3. ASSET ALLOCATION

Stocks: 60%
Bonds: 30%
Cash: 5%
Alternatives: 5%

4. RISK ANALYSIS

Sharpe Ratio: 1.2
Volatility: 12.5%
Maximum Drawdown: -15.3%

5. BENCHMARK COMPARISON

S&P 500 Return: 9.2%
Relative Performance: -0.7%

6. RECOMMENDATIONS

 Maintain current bond allocation for stability while keeping sufficient equity exposure for long-term growth.

DISCLAIMER

Past performance does not guarantee future results. This report is for informational purposes only.


